<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/main/CMPUT_663_ReuseRate_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics

In [2]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [32]:
df = pd.read_excel('/content/ReusabilityRate_ML/DataSets/packages.xlsx')
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [33]:
df

,Project,LongName,Name,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL,ReuseRate
0,ch.qos.logback/logback-classic,<root_package>,<root_package>,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0,0
1,ch.qos.logback/logback-classic,ch,ch,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
2,ch.qos.logback/logback-classic,unnamed package,unnamed package,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0,0
3,ch.qos.logback/logback-classic,ch.qos,qos,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
4,ch.qos.logback/logback-classic,ch.qos.logback,logback,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,xerces/xercesImpl,org.apache.xerces.impl.xs.identity,identity,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10,0
2189,xerces/xercesImpl,org.apache.xerces.impl.xs.models,models,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10,0
2190,xerces/xercesImpl,org.apache.xerces.impl.xs.opti,opti,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15,3
2191,xerces/xercesImpl,org.apache.xerces.impl.xs.traversers,traversers,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29,0


In [34]:
df.iloc[:,3:-1]

,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL
0,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0
1,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
2,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0
3,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
4,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10
2189,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10
2190,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15
2191,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29


In [60]:
class ReuseTradML:
  def __init__(self,data,level):
    self.ml_algo_list = ['Polynomial Regression',
                         'MLPRegressor',
                         'BayesianRidge',
                         'SGD Regressor',
                         'XGB Regressor',
                         'Decision Tree Regressor',
                         'ElasticNet',
                         'SVM Regressor',]

    self.metrics = ['RMSE',
                    'MAE']
    self.data_columns = None
    self.level=level

    if self.level == 'package':
      target_scaler = MinMaxScaler()
      df = data.iloc[:,3:-1]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      target_scaler.fit(self.X_np)
      self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'class':
      target_scaler = MinMaxScaler()
      df = data.iloc[:,3:-1]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      target_scaler.fit(self.X_np)
      self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'project':
      self.X_np = np.array(data.iloc[:,4:])
      self.other = data.iloc[:,:4]

    else:
      print('Error, Please try: class, package, or project. Thanks!')
      self.X=None
      self.Y=None
      self.other=None

  def apply_ml(self):
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
  
    metric = []
    metric_count = 0

    level = []
    model = []

    mean_RMSE = []
    stdv_RMSE = []
    mean_MAE = []
    stdv_MAE = []

    for i in range(self.X.shape[1]):
      #print(self.X[i])
      for j in self.ml_algo_list:
        if j == 'Polynomial Regression':
          
          #temp_data = self.X[i]
          level.append(self.level)
          metric.append(self.data_columns[i])

          cnt=0
          RMSE=[]
          MAE=[]
          poly_reg = PolynomialFeatures(degree=7)
          temp_data = poly_reg.fit_transform(self.X_np[i].reshape(-1, 1))
          pol_reg = LinearRegression()
          model.append('Polynomial Regression')

          for train_index, test_index in kf.split(temp_data):
            cnt = cnt+1
            print('Fold: ',cnt)
            X_train, X_test = temp_data[train_index], temp_data[test_index]
            y_train, y_test = self.Y[train_index], self.Y[test_index]

            pol_reg.fit(X_train,y_train)
            y_pred = pol_reg.predict(X_test)
            RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
            MAE.append(mean_absolute_error(y_test,y_pred))
          
          mean_RMSE.append(statistics.mean(RMSE))
          stdv_RMSE.append(statistics.pstdev(RMSE))
          mean_MAE.append(statistics.mean(MAE))
          stdv_MAE.append(statistics.pstdev(MAE))

        else:
          

          #['Polynomial Regression','MLPRegressor','BayesianRidge','SGD Regressor','XGB Regressor','Decision Tree Regressor','ElasticNet','SVM Regressor',]
          if j == 'MLPRegressor':
            reg = MLPRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          #if j == 'Logistic Regression':
            #reg = LogisticRegression()
            #print('Metric: ',self.data_columns[i])
            #print('Level: ',self.level)
            #print('Model: ',j)

          if j == 'BayesianRidge':
            reg = BayesianRidge()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          if j == 'SGD Regressor':
            reg = SGDRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          if j == 'XGB Regressor':
            reg = XGBRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          if j == 'Decision Tree Regressor':
            reg = DecisionTreeRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          if j == 'ElasticNet':
            reg = ElasticNet()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          if j == 'SVM Regressor':
            reg = SVR()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)

          temp_data = self.X[i].reshape(-1, 1)
          level.append(self.level)
          metric.append(self.data_columns[i])

          cnt=0
          RMSE=[]
          MAE=[]
          for train_index, test_index in kf.split(temp_data):
            cnt = cnt+1
            print('Fold: ',cnt)
            X_train, X_test = temp_data[train_index], temp_data[test_index]
            y_train, y_test = self.Y[train_index], self.Y[test_index]

            reg.fit(X_train,y_train)
            y_pred = reg.predict(X_test)
            RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
            MAE.append(mean_absolute_error(y_test,y_pred))

          mean_RMSE.append(statistics.mean(RMSE))
          stdv_RMSE.append(statistics.pstdev(RMSE))
          mean_MAE.append(statistics.mean(MAE))
          stdv_MAE.append(statistics.pstdev(MAE))
      

       

      #break
    metric = pd.DataFrame(metric,columns=['Metric'])
    level = pd.DataFrame(level,columns=['Level'])
    model = pd.DataFrame(model,columns=['Model'])

    mean_RMSE = pd.DataFrame(mean_RMSE,columns=['RMSE'])
    stdv_RMSE = pd.DataFrame(stdv_RMSE,columns=['STD RMSE'])
    mean_MAE = pd.DataFrame(mean_MAE,columns=['MAE'])
    stdv_MAE = pd.DataFrame(stdv_MAE,columns=['STD MAE'])

    res_df = pd.concat([metric,level,model,mean_RMSE,stdv_RMSE,mean_MAE,stdv_MAE],axis=1)

    return res_df

In [61]:
rtm=ReuseTradML(df,level='package')
res=rtm.apply_ml()
res

Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  AD
Level:  package
Model:  MLPRegressor
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  AD
Level:  package
Model:  BayesianRidge
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  AD
Level:  package
Model:  SGD Regressor


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when 

Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  AD
Level:  package
Model:  XGB Regressor
Fold:  1
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  2
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  3
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Fold:  4
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  5
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  6
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  7
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  8
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  9
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  10
[19:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Metric:  AD
Level:  package
Model:  Decision Tr

...

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when 

Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  NCL
Level:  package
Model:  BayesianRidge
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  NCL
Level:  package
Model:  SGD Regressor
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  NCL
Level:  package
Model:  XGB Regressor
Fold:  1
[19:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  2
[19:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  3
[19:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  4
[19:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold:  5
[19:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:lin

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Fold:  10
[19:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Metric:  NCL
Level:  package
Model:  Decision Tree Regressor
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  NCL
Level:  package
Model:  ElasticNet
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10
Metric:  NCL
Level:  package
Model:  SVM Regressor
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Fold:  10


,Metric,Level,Model,RMSE,STD RMSE,MAE,STD MAE
0,AD,package,Polynomial Regression,0.034253,0.100869,0.024345,0.071333
1,AD,package,Polynomial Regression,0.026531,0.007998,0.025922,0.007863
2,AD,package,Polynomial Regression,0.000409,0.000509,0.000351,0.000362
3,AD,package,Polynomial Regression,0.000332,0.000559,0.000263,0.000401
4,AD,package,Polynomial Regression,0.000431,0.000496,0.000376,0.000351
...,...,...,...,...,...,...,...
139,NCL,package,NaN,0.000341,0.000557,0.000269,0.000392
140,NCL,package,NaN,0.000443,0.000505,0.000400,0.000388
141,NCL,package,NaN,0.000346,0.000534,0.000271,0.000388
142,NCL,package,NaN,0.000410,0.000511,0.000358,0.000360


In [43]:
rtm.data_columns.shape

(18,)